In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

# Introduction to SQL (Structured Query Language)

##### Version 0.1

***

By Scott Coughlin (Northwestern IT Research Computing Services)  
20 July 2022

[Session 15](https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions/tree/main/Sessions/Session15) is primarily concerned with handling our data with efficiency.

Ideally, for any and every task we want to desire solutions that operate *faster*. 

This can be accomplished many different ways:

$~~~~~~$build algorithms that execute faster

$~~~~~~$spread calculations over many different computers simultaneously

$~~~~~~$find a compact storage solution for the data so it can be accessed more quickly

In our introduction to SQL we will start with simple queries of existing tables, and discuss creating your own tables using `pandas` as a challenge problem. 

## Problem 1) IMDb Data

Throughout the session we will use information from the [Internet Movie Database (IMDb)](https://www.imdb.com/) to illustrate various principles regarding data storage.

For this notebook, we will use a [google sheets](https://docs.google.com/spreadsheets/d/1B-C7uJFrVNGpAXsGE6_xymfFVSKhwnIsI_RewkkmGa0/edit?usp=sharing) spreadsheet to examine this data (later in the week we will explore the same data stored in a database). 

A quick note on the provenance of this data. The files we have used to populate this data set are from [this website](https://relational.fit.cvut.cz/dataset/IMDb) and it may not be a list of every single movie on IMDb (there are no movies after 2004).

**Problem 1a**

Open the google sheet.$^\dagger$ What information is stored in the `movies` sheet of this file? 

How many movies are listed? 

$^\dagger$Note – the link points to a "view" of the data, you may find it useful to copy the file so that you have write access. 

*Write your answer here*

**Problem 1b**

What information is stored in the `directors` sheet? 

How many directors are there? 

*Write your answer here*

**Problem 1c**

What information is stored in the `movies_directors` sheet?

How many rows are there? Does this make sense? Why?

*Write your answer here*

**Problem 1d**

Confirm your column identifications in **1c** by finding your favorite movie and making sure `movies_directors` correctly matches it with the proper director.

*Write your answer here*

**Problem 1e**

What information is stored in the `movies_genres` sheet?

How many rows are there? Does this make sense? Why?

*Write your answer here*

## Problem 2) Connections

We started this exercise with a goal of being efficient. And yet, the data have been organized across 4 different files (each sheet is effectively a unique csv file).  

**Problem 2a**

If we wanted to store all the data in a single sheet, how many rows would we need?

*write your answer here*

**Problem 2b**

Supposing we did convert everything to a single sheet – how many columns would you need to store all the information in a single sheet? 

*Think about this for at least 30 sec, but you do not have to write down a full answer*

**Problem 2c**

Which has fewer total entries (the combination of a column and a row makes 1 entry) – all the data in one sheet, or the data spread across 4 sheets? 

*write your answer here*

What we see here is that it is very inefficient/expensive to record "nothing". 

Suppose there is a director that only has one film credit. In the single sheet solution, we need to record that `movie2` = `NULL`, and `movie3` and so on. This information has a cost (storage), AND, there is essentially nothing learned from said information. 

The reason the data have been organized into several sheets (or shall we call them "data tables"; yeah! let's call them "tables") is that it allows us to store far less information. There are some columns that we otherwise would not need (in a single table there is no need for an `id`, whereas the multi-table solution has at least one id in every table). 

The `id` however is very powerful, as it is what allows the connection between the different tables.

**Problem 2d**

What is the release year for every movie directed by Martin Scorcese? 

*Hint* – do not spend more than 10 min working on this problem (you probably should not even spend a full 10 min). 

*write your answer here*

## Problem 3) Leverage

Now that we know why the data has been organized in this way, we can leverage this unique structure in order to learn interesting properties of the data. 

**Problem 3a**

As a follow-up to a previous problem, according the the IMDb data, which director has directed the most movies?

*Hint* – there are probably several ways to figure this out, but I'll note that google sheets provides the ability to calculate aggregated statistics *after* the data have been grouped in a single category. This type of operation exists on many different platforms, and is typically called a `group by`. 

For google sheets, a `group by` is performed inside the built in `QUERY` function. For example, if you were to type the following into a cell:

`=QUERY(A1:D25, "select B, AVG(D) group by B", 1)`

this would look at all the data in columns A--D, and between rows 1--25, it would then select values in B and "group the data" (i.e., suppose this column listed city names and column D listed temperatures, then every entry for the same city would be "grouped"), then aggregate statistics are caluclated on D (in this example, the average temperature for the city would be calculated). The `1` at the end indicates a single header row to those being searched.

The results would be a column saying city and a column saying temperature, and then the average temp in each city would be recorded. 

*Hint 2* – run your query on only a few rows first to make sure it works in the expected fashion.

*write your answer here*

We now know who (according to IMDb) has directed the most movies ever.

You should feel a little unsatisfied with this solution, however. While we had a clever way to count the movies by director, we still had to click over to another sheet and scroll (or `cntl+f`) to find the name of the director.

It would be far more satisfying to use a single command that returns director name *and* number of movies. 

To do this in a single command requires a `JOIN` (i.e., data from two different tables/sheets must be combined). 

Sadly, joins are not possible in google sheets$^\dagger$ (or at least I've spent a lot of time trying, and I have not found anything close to a useful solution even for the moderate amount of data included in this project).

$^\dagger$If you know how to do this let me know.

To understand why the join operation is so powerful, consider the following question:

Which drama directed by Martin Scorsese has the highest IMDb rating? 

This question is not that complex, but, we need to do the following to solve it with our spreadsheet: 
  1. look up the Scorsese `director_id`
  2. use `movies_directors` to look up the movies Scorsese has directed
  3. use `movies_genres` to figure out which of those movies are dramas
  4. select and sort the subset of Scorsese drama `movies` based on score

That's a lot of clicking around and running multiple operations. 

We will see several other methods later this week that are capable of `JOIN`ing tables so that this can be accomplished quickly.

**Problem 3b**

In which year were the most movies made according to IMDb?

*write your answer here*

## Challenge Problem) Algorithmic Speed

For the challenge problem we will consider a different type of efficiency – algorithms.

We will demonstrate the types of improvements that are possible even for simple algorithms, like "sorting".

Both `python` and `numpy` have build-in functions that can sort a list or array with a great deal of efficiency. In practice, I would never recommend writing your own function to this end, but we will demonstrate how much time can be saved via "clever" implementations of straightforward ideas. 

**Challenge a**

Write a "pure" python function to sort the numbers in a list. Restrict yourself to simple list operations (i.e., do not use `np.sort()`. 

*Hint* – we can sort a list by creating a new list, `sort_list`, and looping over all elements in the original list and inserting each element in the position where it is greater than the previous value and less than or equal to the next value in `sorted`.

In [ ]:
# complete

**Challenge b**

Confirm your algorithm works by running it on the following list: [4, 1, 6, 2, 4, 7, 9, 3, 8]

In [ ]:
# complete ([4, 1, 6, 2, 4, 7, 9, 3, 8])

**Challenge c**

Create a list with 10000 random numbers. Time how long it takes to sort that list with your algorithm.

In [ ]:
import time

np.random.seed(1662)

haha = list(np.random.randn(10000))

tstart = time.time()
# complete
tend = time.time()

print('It takes {:.4f} s to sort this list'.format(tend-tstart))

10000 elements is not that many, and 3 s is a lot of time. In this case the speed of the algorithm is quite slow because we have a nested for loop. As the sorted array becomes larger, we have to again and again consider all the same numbers while looking for the "right" place to insert our value. 

We can achieve dramatic improvements with a "divide and conquer" strategy. In particular, we will use the [`quicksort`](https://en.wikipedia.org/wiki/Quicksort) algorithm to avoid looking at the same numbers again and again. 

In brief, `quicksort` does the following: pick a single element of an array/list (called the "pivot"), move all values less than the pivot to its right, move all values greater than the pivot to its left.

The pivot is now in its correct position within the array. We can now perform the same operation on the left and on the right to recursively move everything into the correct order. Unlike the "simple" method, we do not need to "look at" the same values again and again and again.

**Challenge d**

Write a recursive `quicksort` algorithm to sort values in a list/array. 

*Hint* – use the last element of the array as the "pivot" each time you separate the data into high and low values.

In [ ]:
# complete

**Challenge e**

Confirm your algorithm works by sorting the short list [4, 1, 6, 2, 4, 7, 9, 3, 8].

In [ ]:
short_list = [4, 1, 6, 2, 4, 7, 9, 3, 8]
# complete

**Challenge f**

Time how long it take to sort the random number array using `quicksort`.

In [ ]:
meh = haha.copy()
tstart = time.time()
# complete
tend = time.time()

print('It takes {:.4f} s to sort this list'.format(tend-tstart))

A factor of 100 improvement! 

**Challenge g**

Time how long it takes to sort the random number array using `np.sort()`. 

In [ ]:
meh = haha.copy()

tstart = time.time()
# complete
tend = time.time()

print('It takes {:.4f} s to sort this list'.format(tend-tstart))

While your solution was quite clever, under the hood `numpy` is still another factor of 5 faster! 

The upside from this exercise – every time you find yourself creating a for loop, ask the question do I really need a for loop. For the most part you will always be working with software that is already optimized but the for loops that you write may not be...